In [1]:

import tensorflow as tf 
import numpy as np 
from matplotlib import pyplot as plt 
import cv2

#### - LOAD A MODEL

In [2]:
#interpreter = tf.lite.Interpreter(model_path="Model/lite-model_movenet_singlepose_lightning_3.tflite")
interpreter = tf.lite.Interpreter(model_path="Model/lite-model_movenet_singlepose_thunder_3.tflite")
interpreter.allocate_tensors()

In [3]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

[{'name': 'serving_default_input:0', 'index': 0, 'shape': array([  1, 256, 256,   3], dtype=int32), 'shape_signature': array([  1, 256, 256,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


#### - MAKE DETECTIONS

## <font color=yellow> 3.Draw Keypoints

#### - hos squeez function works

In [4]:

import numpy as np

# Create an array with a single-dimensional entry in its shape
x = np.array([[[0], [1], [2]]])
print(x.shape) # (1, 3, 1)

# Remove the single-dimensional entry from the shape of the array
y = np.squeeze(x)
print(y.shape) # (3,)
print(y)

(1, 3, 1)
(3,)
[0 1 2]


#### - function render 

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c= frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    for kp in shaped:
        ky,kx,kp_conf = kp

        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

## <font color=yellow> 4.Draw Edges

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}


In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [13]:
cap = cv2.VideoCapture(0) # the number said the webcam you will use, more webcams more nymbers which you can play 

cv2.startWindowThread()
while cap.isOpened():
    ret, frame = cap.read()

    # Reshape image: because model expect a input image of 192x192x3
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
    #img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
        #% tf.image.resize_with_pad:
        #@ is a tensorflow function which resize a image adding padding if needed to mantaing the proportion of the img

        #% np.expand_dims(img, axis=0):(encapsulate in another array, axis=0 number of new dimenstion + 1)
        #@ is the first argument we pass to that function, which adds a new axis to the image array because the tf
        #@ function expect a 4D array because the first argument is the batch size and given that is a single image 
        #@ we only put 1 in that argument

        #@ the last two argument are the height and width arguments 

    # we cast our image (we convert the data type to another data type), given that the model expect 
    # a image in that specific data type 
    input_image = tf.cast(img, dtype=tf.float32)


    #Set up input ann output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Make predictions

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
        #@ set the value of a tensor in the interpreter , the first argument indicates the position and second what will be in to it 
    interpreter.invoke()
        #@ use this method to run inference in the input data, that means we will use the image given to put in the model lite 
    keypoints_with_scores = interpreter.get_tensor(output_details[0]["index"])
        #@ get_tensor: is a copy of the tensor data 
        #@ that index extract from the output_detail will hold the index in which our output resolt is in the interpretor 
    #print(keypoints_with_scores)

    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores[0][0], 0.4)


    cv2.imshow("MoveNet lighting", frame)
    cv2.resizeWindow('image', 600, 1200)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break 

                
    

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1